In [1]:
import pandas as pd
from openai import OpenAI
import ast
import tqdm
from Py_Files import credentials

# CHATGPT LLM

In [ ]:
client = OpenAI(api_key=credentials.OPENAI_API_KEY)

tickers_with_q_df = pd.read_csv('/Users/joeybortfeld/Downloads/tickers_with_q.csv')


name_list = list(zip(tickers_with_q_df['Name1'].tolist(), tickers_with_q_df['Name2'].tolist()))


def ask_chatgpt(prompt):

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": my_prompt
            }
        ]
    )

    return completion
    return completion.choices[0].message.content



all_results = []
error_list = []
print('name count:', len(name_list))

for this_name in tqdm.tqdm(name_list[:15]):

    this_name1, this_name2 = this_name

    try:
        my_prompt = f"Did {this_name}1, which may also be known as {this_name2}, ever declare bankruptcy? Answer with only a tuple of three elements. THe first element is 'yes' or 'no'. The second element is the date of the bankruptcy if yes otherwise None. The third element is a short text paragraph describing the bankruptcy not to exceed four sentences but if there was no bankruptcy then return an explanation. The paragraph should be wrapped in double quotes. If you do not know the answer return with a tuple of form ('unknown', 'None', 'NA')"
        result = ask_chatgpt(my_prompt)


        # SPECIAL CASE: the answer is returned as a string tuple. Reformat to actual tuple
        # result = ast.literal_eval(result)
        all_results.append([this_name1, this_name2, result])

    except:
        print('--error on:', this_name)
        print(result)
        error_list.append(this_name)


all_results2 = []
for i in range(len(all_results)):
    
    try:

        e = all_results[i]
        name1 = e[0]
        name2 = e[1]

        # parse the response from ChatGPT which is a string and convert it into a tuple
        result_tuple = ast.literal_eval(e[2])
        
        result_tuple = [name1, name2] + list(result_tuple)
        all_results2.append(result_tuple)
    except:
        print(f'error on {i}')

df2 = pd.DataFrame(all_results2, columns=['name1', 'name2', 'bankruptcy', 'date', 'description'])

# df2.to_csv('/Users/joeybortfeld/Downloads/sample_bankruptcies_chatgpt.csv', index=False)
df2

In [6]:
this_name1 = "microsoft"
this_name2 = 'Microsoft Corp'
my_prompt = f"Did {this_name1}, which may also be known as {this_name2}, ever declare bankruptcy? Answer with only a tuple of three elements. THe first element is 'yes' or 'no'. The second element is the date of the bankruptcy if yes otherwise None. The third element is a short text paragraph describing the bankruptcy not to exceed four sentences but if there was no bankruptcy then return an explanation. The paragraph should be wrapped in double quotes. If you do not know the answer return with a tuple of form ('unknown', 'None', 'NA')"
result = ask_chatgpt(my_prompt)

In [ ]:
# ompletion.choices[0].message.content
ast.literal_eval(result.choices[0].message.content)

In [ ]:
# SAMPLE CODE FROM OPENAI TUTORIAL
# OpenAI: https://platform.openai.com

from openai import OpenAI
client = OpenAI(OPENAI_API_KEY)

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)

# CLAUDE LLM

In [ ]:



from anthropic import Anthropic

def classify_sector_from_description_anthropic(description, classes):

    client = Anthropic(api_key=credentials.ANTHROPIC_API_KEY)

    response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    messages=[
        {"role": "user", 
         "content": f'''Given the following company description, classify the company into one of these 
         sectors: {', '.join(sector_classes)}.\n\nDescription: {this_des}\n\nSector:\n\n
         Provide your response as a tuple where the first element is one of the sectors provided,
         the second element is a measure of confidence in the classification from 0 to 100 and the third
         element is your next best classification. Your response should just be the tuple and no other text'''}
        ]
    )

    text_response = response.content[0].text
    return ast.literal_eval(text_response)

results = []
for this_row in df_company_profiles.iterrows():

    if this_row[0] % 50 == 0:
        print('--', this_row[0])

    this_des = this_row[1]['description']
    this_symbol = this_row[1]['symbol']
    this_name = this_row[1]['name']
    this_industry = this_row[1]['industry']
    this_sector = this_row[1]['sector']


    # censor the company name from the description to prevent leakage 
    this_des = this_des.replace(this_name, 'XXX')
    try:
        llm_result = classify_sector_from_description_anthropic(this_des, sector_classes)
        this_classification = llm_result[0]
        this_confidence = llm_result[1]
        this_next_best = llm_result[2]

        this_classification = [this_symbol, this_classification, this_confidence, this_next_best]
        results.append(this_classification)
    except:
        print(f'error on {this_symbol}')

results = pd.DataFrame(results, columns=['symbol', 'classification', 'confidence', 'next_best'])
results = df_company_profiles.merge(results, how='outer', on='symbol')
print((results['sector'] == results['classification']).mean())

# Bankruptcy Search

input = ['104 Corporation', '104 Corp.']
